# LEGO Detection

### Libraries

In [ ]:
# import libraries

import os
import cv2
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from google.colab import drive
from xml.etree import ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm import tqdm
from torchvision.transforms import functional as F

### Data preparation

In [ ]:
# mount drive on colab notebook

drive.mount('/content/drive')

In [ ]:
# unzip data files

!unzip "/content/drive/MyDrive/02 - tagged1.zip" -d "/content"

In [ ]:
# major variables

photos_dir = '/content/photos'
renders_dir = '/content/renders'

# photos_dir = 'content/photos' # running locally
# renders_dir = 'content/renders' # running locally

In [ ]:
def load_data(data_dir):
    '''
    Returns a list of images and labels for each image
    '''
    image_paths = []
    num_legos = []
    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                n = int(subdir.split(os.sep)[-1])
                image_paths.append(os.path.join(subdir, file))
                num_legos.append(n)
    combined = list(zip(image_paths, num_legos))
    combined.sort()
    image_paths, num_legos = zip(*combined)
    image_paths = np.asarray(image_paths)
    num_legos = torch.Tensor(num_legos).to(torch.int64)
    return image_paths, num_legos

In [ ]:
def parse_xml(xml_file):
    '''
    Read the xml file and return the bounding box coordinates
    '''
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bounding_boxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])
    return bounding_boxes

In [ ]:
def parse_all_xml(image_paths):
    '''
    Parse all XML files corresponding to the given image paths.
    '''
    bounding_boxes = []
    for img_path in image_paths:
        xml_path = img_path.replace('.jpg', '.xml')
        bounding_boxes.append(parse_xml(xml_path))
    return bounding_boxes

In [ ]:
# load data

image_paths, num_legos = load_data(photos_dir)

In [ ]:
# parse bounding boxes for all images

bounding_boxes = parse_all_xml(image_paths)

In [ ]:
# class distribution in overall data

plt.hist(num_legos, bins=range(1, max(num_legos)), align='left', rwidth=0.8)
plt.xlabel('Number of Legos')
plt.ylabel('Frequency')
plt.title('Legos Distribution')
plt.show()

In [ ]:
# work with defined train test split

train_test_split = np.genfromtxt('/content/drive/MyDrive/train_test_split.csv', delimiter=',', dtype=None, encoding=None)
# train_test_split = np.genfromtxt('content/train_test_split.csv', delimiter=',', dtype=None, encoding=None) # running locally

train_test_ids = {
    'train': [],
    'test': []
}
for index, row in enumerate(train_test_split):
    if row[1] == '1':
      train_test_ids['test'].append(index - 1)
    elif row[1] == '0':
      train_test_ids['train'].append(index - 1)

len(train_test_ids['train']), len(train_test_ids['test'])

In [ ]:
# class distribution in training data

num_legos_train = num_legos[train_test_ids['train']]
plt.hist(num_legos_train, bins=range(1, max(num_legos_train)), align='left', rwidth=0.8)
plt.xlabel('Number of Legos')
plt.ylabel('Frequency')
plt.title('Legos Distribution (Training)')
plt.show()

In [ ]:
# undersampling of larger classes in training data

indices1 = []
indices2 = []

for i in train_test_ids['train']:
    if num_legos[i] == 1:
        indices1.append(i)
    elif num_legos[i] == 2:
        indices2.append(i)
        
np.random.shuffle(indices1, )
np.random.shuffle(indices2, )

leftovers1 = indices1[100:]
leftovers2 = indices2[100:]

for i in leftovers1:
    train_test_ids['train'].remove(i)
for i in leftovers2:
    train_test_ids['train'].remove(i)

num_legos_train = num_legos[train_test_ids['train']]
plt.hist(num_legos_train, bins=range(1, max(num_legos_train)), align='left', rwidth=0.8)
plt.xlabel('Number of Legos')
plt.ylabel('Frequency')
plt.title('Legos Distribution (After Undersampling)')
plt.show()

In [ ]:
# oversampling of smaller classes in training data - augmentation

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor()
])

# TODO: apply augmentation to classes

In [ ]:
# validation set

indices = train_test_ids['test']
np.random.shuffle(indices, )

test_size = 0.4 * len(indices)
split = int(np.floor(test_size))
train_test_ids['valid'], train_test_ids['test'] = indices[split:], indices[:split]

len(train_test_ids['train']), len(train_test_ids['valid']), len(train_test_ids['test'])

In [ ]:
class LegosDataset(Dataset):
    '''
    Dataset class for the legos dataset
    '''
    def __init__(self, images_filenames, num_legos, bounding_boxes, transform=None):
        self.images_filenames = images_filenames
        self.transform = transform
        self.labels = num_legos
        self.bounding_boxes = bounding_boxes

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        label = self.labels[idx]
        bounding_boxes = self.bounding_boxes[idx]
        image = cv2.imread(image_filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_height, original_width = image.shape[:2]
        if self.transform is not None:
            image = self.transform(image)
        scale_w = 224.0 / original_width
        scale_h = 224.0 / original_height
        scaled_boxes = []
        for box in bounding_boxes:
            x_min, y_min, x_max, y_max = box
            x_min = int(x_min * scale_w)
            y_min = int(y_min * scale_h)
            x_max = int(x_max * scale_w)
            y_max = int(y_max * scale_h)
            scaled_boxes.append([x_min, y_min, x_max, y_max])
        target = {
            'boxes': torch.tensor(scaled_boxes, dtype=torch.float32),
            'labels': torch.ones((label,), dtype=torch.int64)
        }
        return image, target

In [ ]:
# bounding boxes for train, valid, and test sets

train_boxes = [bounding_boxes[i] for i in train_test_ids['train']]
valid_boxes = [bounding_boxes[i] for i in train_test_ids['valid']]
test_boxes = [bounding_boxes[i] for i in train_test_ids['test']]

In [ ]:
# train, valid and test sets

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = LegosDataset(image_paths[train_test_ids['train']], num_legos[train_test_ids['train']], train_boxes, transform=transform)
valid_dataset = LegosDataset(image_paths[train_test_ids['valid']], num_legos[train_test_ids['valid']], valid_boxes, transform=transform)
test_dataset = LegosDataset(image_paths[train_test_ids['test']], num_legos[train_test_ids['test']], test_boxes, transform=transform)

batch_size = 32
num_workers = 2

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

### Model definition

In [ ]:
# get cpu or gpu device for training

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# faster r-cnn model

faster_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = faster_rcnn.roi_heads.box_predictor.cls_score.in_features
num_classes = 2
faster_rcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# put model in device

model = faster_rcnn.to(device)

model

### Model training

In [ ]:
def epoch_iter(dataloader, model, optimizer=None, is_train=True):
    '''
    Perform one epoch of training/validation/testing
    '''
    if is_train:
        assert optimizer is not None, "When training, please provide an optimizer"
    num_batches = len(dataloader)
    model.train() if is_train else model.eval()
    total_loss = 0.0
    with torch.set_grad_enabled(is_train):
        for batch, (images, targets) in enumerate(tqdm(dataloader)):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            if is_train:
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
            total_loss += losses.item()
    avg_loss = total_loss / num_batches
    return avg_loss

In [ ]:
def train(model, model_name, num_epochs, train_dataloader, validation_dataloader, optimizer, train_history=None, val_history=None):
    '''
    Train the model
    '''
    if train_history is None:
        train_history = {'loss': []}
    if val_history is None:
        val_history = {'loss': []}
    best_val_loss = np.inf
    print("Start training...")
    for t in range(num_epochs):
        print(f"Epoch {t+1}/{num_epochs}")
        train_loss = epoch_iter(train_dataloader, model, optimizer, is_train=True)
        print(f"Train loss: {train_loss:.3f}")
        val_loss = epoch_iter(validation_dataloader, model, is_train=False)
        print(f"Validation loss: {val_loss:.3f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_dict = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': t
            }
            torch.save(save_dict, model_name + '_best_model.pth')
        save_dict = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': t
        }
        torch.save(save_dict, model_name + '_latest_model.pth')
        train_history['loss'].append(train_loss)
        val_history['loss'].append(val_loss)
    print("Finished")
    return train_history, val_history

In [ ]:
# training and validation history

train_history = {'loss': []}
val_history = {'loss': []}

In [ ]:
# freeze all layers

for param in model.parameters():
    param.requires_grad = False

In [ ]:
def unfreeze_layers(model, layers_to_unfreeze):
    '''
    Unfreeze the specified layers in the model
    '''
    for name, param in model.named_parameters():
        if any(layer in name for layer in layers_to_unfreeze):
            param.requires_grad = True

In [ ]:
# train head

unfreeze_layers(model, ['roi_heads.box_predictor'])

optimizer = torch.optim.Adam(model.roi_heads.box_predictor.parameters(), lr=1e-4)
num_epochs = 5

train_history, val_history = train(
                                model,
                                'lego_detector_head',
                                num_epochs,
                                train_dataloader,
                                valid_dataloader,
                                optimizer,
                                train_history,
                                val_history
                            )

In [ ]:
# train head + RPN

rpn_layers = [layer[0] for layer in list(model.named_parameters()) if 'rpn' in layer[0]]
unfreeze_layers(model, rpn_layers)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
num_epochs = 5

train_history, val_history = train(
                              model,
                              'lego_detector_rpn',
                              num_epochs,
                              train_dataloader,
                              valid_dataloader,
                              optimizer,
                              train_history,
                              val_history
                            )

In [ ]:
# train head + RPN + backbone

backbone_layers = [layer[0] for layer in list(model.named_parameters()) if 'backbone' in layer[0]]
unfreeze_layers(model, backbone_layers)
for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 10

train_history, val_history = train(
    model,
    'lego_detector_final',
    num_epochs,
    train_dataloader,
    valid_dataloader,
    optimizer,
    train_history,
    val_history
  )

### Training evolution analysis

In [ ]:
def plotTrainingHistory(train_history, val_history):
    '''
    Plot the training history of the model
    '''
    plt.subplot(1, 1, 1)
    plt.title('Loss')
    plt.plot(train_history['loss'], label='train')
    plt.plot(val_history['loss'], label='val')
    plt.legend(loc='best')

    plt.tight_layout()
    plt.legend(loc='best')
    plt.show()

In [ ]:
# visualize training history

plotTrainingHistory(train_history, val_history)

### Model testing

In [ ]:
# load best model

model = faster_rcnn.to(device)
checkpoint = torch.load('lego_detector_best_model.pth')
model.load_state_dict(checkpoint['model'])

In [ ]:
# evaluate model on test data

test_loss = epoch_iter(test_dataloader, model, is_train=False)
print(f"Test loss: {test_loss:.3f}")

In [ ]:
def show_predictions(model, dataloader):
    '''
    Display images along with their true and predicted bounding boxes
    '''
    model.eval()
    with torch.no_grad():
        for images, targets in dataloader:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            for img, output, target in zip(images, outputs, targets):
                img = img.cpu().numpy().transpose((1, 2, 0))
                img = F.to_pil_image(torch.tensor(img))
                fig, ax = plt.subplots(1, figsize=(12, 9))
                ax.imshow(img)
                for box in target['boxes']:
                    box = box.cpu().numpy()
                    rect = patches.Rectangle(
                        (box[0], box[1]), 
                        box[2] - box[0], 
                        box[3] - box[1], 
                        linewidth=2, 
                        edgecolor='g', 
                        facecolor='none'
                    )
                    ax.add_patch(rect)
                for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
                    if score > 0.5:
                        box = box.cpu().numpy()
                        rect = patches.Rectangle(
                            (box[0], box[1]), 
                            box[2] - box[0], 
                            box[3] - box[1], 
                            linewidth=2, 
                            edgecolor='b', 
                            facecolor='none'
                        )
                        ax.add_patch(rect)
                        ax.text(box[0], box[1], f'{label.item()}: {score:.2f}', bbox=dict(facecolor='yellow', alpha=0.5))
                plt.title(f'True (Green) and Predicted (Blue) Bounding Boxes')
                plt.axis('off')
                plt.show()

In [ ]:
# view predictions

show_predictions(model, test_dataloader)